In [21]:
%matplotlib inline
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import pathlib

In [51]:
datas = []
entries = pathlib.Path('data/')

for entry in entries.iterdir():
    data = np.genfromtxt(entry, delimiter=',')
    mean_data, std_data = np.mean(data, axis=0), np.std(data, axis=0)
    normalize_data = (data - mean_data)/std_data
    data = torch.from_numpy(normalize_data)
    datas.append(data)